In [29]:
# Basic import
import os
import sys
import json
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2

In [2]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

## 2019 movies

In [12]:
PATH = './SplitTrailers_crop/'
order = os.listdir(PATH)
order.sort()
len(order)

97

## 2018 movies

In [ ]:
df_onehot = pd.read_csv('./csv/filename2genreMat_2018.csv')
df_onehot

In [ ]:
trailers = [filename.split('.')[0] for filename in df_onehot['filename'].tolist()]
trailers[:10]

# A_AverageShotLength

In [13]:
A_li = []
for movie in order: # for 2019
    print('Running:', movie)
    
    sceneList = [int(name.split('_')[1]) for name in os.listdir(PATH + movie)]
    sceneList.sort()
    sceneCount = sceneList[-1]
    
    framePath = PATH + movie + '/scene_' + str(sceneCount) + '/frames/'
    frameList = [int(name.split('.')[0]) for name in os.listdir(framePath)]
    frameList.sort()
    frameCount = frameList[-1]
    
    avg = (frameCount/sceneCount)
    print(frameCount, sceneCount, avg)
    
    A_li.append(avg)

Running: 47metersdown
2854 82 34.80487804878049
Running: adogsjourneymovie
4052 117 34.63247863247863
Running: aftermathmovie
3712 104 35.69230769230769
Running: aftermovie
13695 235 58.276595744680854
Running: alitamovie
3650 102 35.78431372549019
Running: angelhasfallen
3715 102 36.42156862745098
Running: angrybirdsmovie
4158 120 34.65
Running: annabellemovie
3456 66 52.36363636363637
Running: annamovie
3374 89 37.91011235955056
Running: apollo11movie
2693 41 65.6829268292683
Running: arcticmovie
4067 56 72.625
Running: artofracingmovie
3189 99 32.21212121212121
Running: avengers
3504 55 63.70909090909091
Running: beachbummovie
3600 107 33.64485981308411
Running: bernadettefilm
3786 108 35.05555555555556
Running: blindedbythelightmovie
3444 101 34.0990099009901
Running: booksmart
5216 80 65.2
Running: breakthroughmovie
4834 113 42.7787610619469
Running: brianbanksmovie
3189 102 31.264705882352942
Running: brightburnmovie
3933 66 59.59090909090909
Running: brittanyrunsmov
3696 108 34.

In [14]:
write_json(A_li, './json/A_li.json')
len(A_li)

97

# B_ColorVariance
* https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html#color_convert_rgb_luv
* https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.det.html

In [15]:
B_li = []
for movie in order: # for 2019
    rho = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' + scene + '/frames/'
        frameList = [int(name.split('.')[0]) for name in os.listdir(framePath) if not name.startswith('.')]
        frameList.sort()
        keyframe = framePath + '/' + str(frameList[len(frameList)//2]) + '.jpg'
        img = cv2.imread(keyframe)
        luv = cv2.cvtColor(img, cv2.COLOR_BGR2Luv)
        l = luv[:,:,0].ravel()
        u = luv[:,:,1].ravel()
        v = luv[:,:,2].ravel()
        x = np.asarray((l, u, v))
        cov = np.cov(x)
        det = np.linalg.det(cov)
        rho.append(det)
    
    avg = (sum(rho)/len(rho))
    print(sum(rho), len(rho), avg)
    
    B_li.append(avg)

Running: 47metersdown
976987572.9134294 82 11914482.596505236
Running: adogsjourneymovie
315403851.6429116 117 2695759.4157513813
Running: aftermathmovie
77256273.03978151 104 742848.7792286684
Running: aftermovie
1334600538.7806818 235 5679151.228853965
Running: alitamovie
262054719.65230814 102 2569163.9181598835
Running: angelhasfallen
123395157.3963219 102 1209756.4450619793
Running: angrybirdsmovie
44389861329.427925 120 369915511.078566
Running: annabellemovie
301126196.4617106 66 4562518.128207736
Running: annamovie
896279007.6099898 89 10070550.647303255
Running: apollo11movie
265294246.3347195 41 6470591.374017549
Running: arcticmovie
23514892.408853084 56 419908.79301523365
Running: artofracingmovie
1009834976.9927531 99 10200353.302957103
Running: avengers
93083652.0974659 55 1692430.0381357437
Running: beachbummovie
1146666270.389741 107 10716507.199904121
Running: bernadettefilm
889222261.5668641 108 8233539.458952446
Running: blindedbythelightmovie
950732930.503401 101 94

In [16]:
write_json(B_li, './json/B_li.json')
len(B_li)

97

# C_MotionContent
* https://blog.gtwang.org/programming/opencv-motion-detection-and-tracking-tutorial/

In [32]:
for movie in order[:1]: # for 2019
#     for scene in os.listdir(PATH + movie)[:5]:
    scene = 'scene_16'
    video  = PATH + movie + '/' + scene + '/clip.avi'
    print(video)

    cap = cv2.VideoCapture(video)
    
    w = int(cap.get(3))
    h = int(cap.get(4))
    
    print(cap, w, h)
    
    # 計算畫面面積
    area = w * h
    
    # 初始化平均影像
    ret, frame = cap.read()
    
    avg = cv2.blur(frame,(4, 4))
    avg_float = np.float32(avg)
    plt.imshow(frame)
    plt.show()

    while(cap.isOpened()):
        print('open')
        # 讀取一幅影格
        ret, frame = cap.read()

        # 若讀取至影片結尾，則跳出
        if ret == False:
            break

        # 模糊處理
#             blur = cv2.blur(frame, (4, 4))

        # 計算目前影格與平均影像的差異值
        diff = cv2.absdiff(frame, avg)

        # 將圖片轉為灰階
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

        # 篩選出變動程度大於門檻值的區域
        ret, thresh = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY)

        # 使用型態轉換函數去除雜訊
        kernel = np.ones((5, 5), np.uint8)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

        # 產生等高線
        cntImg, cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for c in cnts:
            # 忽略太小的區域
            if cv2.contourArea(c) < 2500:
                continue

        # 計算等高線的外框範圍
#             (x, y, w, h) = cv2.boundingRect(c)

        # 畫出外框
#             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # 畫出等高線（除錯用）
        cv2.drawContours(frame, cnts, -1, (0, 255, 255), 2)

        # 顯示偵測結果影像
#             cv2.imshow('frame', frame)

        plt.imshow(frame)
        plt.show()

#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break

        # 更新平均影像
#             cv2.accumulateWeighted(blur, avg_float, 0.01)
        avg = frame #cv2.convertScaleAbs(avg_float)

    cap.release()
    cv2.destroyAllWindows()

./SplitTrailers_crop/47metersdown/scene_16/clip.avi
<VideoCapture 0x7f924d242b10> 1280 720


error: OpenCV(4.1.2) /io/opencv/modules/core/src/matrix.cpp:757: error: (-215:Assertion failed) dims <= 2 && step[0] > 0 in function 'locateROI'


# D_LightingKey

In [20]:
D_li = []
for movie in order: # for 2019
    li = []
    print('Running:', movie)
    
    for scene in os.listdir(PATH + movie):
        framePath = PATH + movie + '/' + scene + '/frames/'
        frameList = [int(name.split('.')[0]) for name in os.listdir(framePath) if not name.startswith('.')]
        frameList.sort()
        keyframe = framePath + '/' + str(frameList[len(frameList)//2]) + '.jpg'
        img = cv2.imread(keyframe)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        value = hsv[:,:,2].ravel()
        mean = np.mean(value)
        std = np.std(value)
        li.append(mean*std)

    avg = sum(li)/len(li)
    print(sum(li), len(li), avg)
    
    D_li.append(avg)

Running: 47metersdown
314822.36706779496 82 3839.2971593633533
Running: adogsjourneymovie
382124.58469297737 117 3266.022091392969
Running: aftermathmovie
334035.01279071113 104 3211.875122987607
Running: aftermovie
746573.9127210833 235 3176.910266898227
Running: alitamovie
290065.6376809985 102 2843.7807615784163
Running: angelhasfallen
375983.7892655306 102 3686.1155810346136
Running: angrybirdsmovie
927207.6518963797 120 7726.730432469831
Running: annabellemovie
91368.09009207874 66 1384.3650013951324
Running: annamovie
406396.7811374262 89 4566.2559678362495
Running: apollo11movie
169031.45605576612 41 4122.7184403845395
Running: arcticmovie
151439.22202876196 56 2704.2718219421777
Running: artofracingmovie
391396.51106689 99 3953.5001117867673
Running: avengers
170629.9833259984 55 3102.363333199971
Running: beachbummovie
493624.1446444189 107 4613.309763031953
Running: bernadettefilm
514559.8719030498 108 4764.443258361573
Running: blindedbythelightmovie
512983.6756476542 101 50

In [21]:
write_json(D_li, './json/D_li.json')
len(D_li)

97

In [22]:
results = pd.DataFrame({
    'Movie': order,
    'A_AverageShotLength': A_li,
    'B_ColorVariance': B_li,
#     'C_MotionContent': C_li,
    'D_LightingKey' : D_li,
})
results

,Movie,A_AverageShotLength,B_ColorVariance,D_LightingKey
0,47metersdown,34.804878,1.191448e+07,3839.297159
1,adogsjourneymovie,34.632479,2.695759e+06,3266.022091
2,aftermathmovie,35.692308,7.428488e+05,3211.875123
3,aftermovie,58.276596,5.679151e+06,3176.910267
4,alitamovie,35.784314,2.569164e+06,2843.780762
...,...,...,...,...
92,usmovie,43.010204,2.260180e+06,3454.562570
93,whatmenwant,32.086957,2.003662e+07,6006.485656
94,wonderparkmovie,27.282443,1.571109e+08,5563.594289
95,xmenmovies,41.549296,4.791049e+06,2953.689532


In [ ]:
# reload computed values
results = pd.DataFrame({
    'Movie': order,
    'A_AverageShotLength': read_json('./json/A_li.json'),
    'B_ColorVariance': read_json('./json/B_li.json'),
    'C_MotionContent': read_json('./json/C_li.json'),
    'D_LightingKey' : read_json('./json/D_li.json'),
})
results